## Описание проекта, загрузка и подготовка данных

### Введение
______
**Название проекта**  

Разработка ML-системы сопоставления товарных позиций с высокой точностью.
_____

**Цель исследования**  

Разработать ML-систему для автоматического сопоставления (мэтчинга) товарных позиций с заданной точностью, предназначенную для поддержки следующих бизнес-задач Заказчика:

- Слияние товарных каталогов от разных поставщиков с устранением дубликатов и несовпадающих наименований

- Инвентаризация и контроль остатков на складах на основе объединённой номенклатуры

- Поиск и подбор аналогов для оперативной замены отсутствующих или снятых с продажи товаров

_______
**Задачи исследования**

- Загрузить и подготовить данные Заказчика
- Провести анализ и необходимую предобработку датасетов с товарами Заказчика и товарами - аналогами
- В связи с небольшим объемом тестовой выборки синтезировать необходимые данные для обучения
- Обучить разные модели и подобрать оптимальный пайплайн из моделей и их параметров для решения задачи
- Проверить данные на тестовой выборке, рассчитать необходимые метрики.
- Разработать сервис/модуль для Заказчика для запуска проекта
_____
**Исходные данные**  

Имеются данные, где основная информация представлена в виде трех датасетов::

- `Список_наших_товаров.csv` — информация о товарах Заказчика:

- `Сметченные_позиции_источник_1` — сметченные товары Заказчика с товарами - аналогами из источника 1

- `Сметченные_позиции_источник_2` — сметченные товары Заказчика с товарами - аналогами из источника 2    
    
*По требованию Заказчика все выводы данных о наименованиях его товаров закомментированы. Абстрактный пример наименования товара:*   
`Пластик плотный-ПВХ для печати, прозрач., 830×1560мм, толщина 2 мм, FIX ECO`
___________
**Оценка результата:**    
будет производиться по метрике `Hits@K` (`accuracy`) - метрика оценки качества сопоставления (matching), которая показывает находится ли правильный ответ (истинные кандидаты) в топ-K отобранных кандидатов.

### Установка и импорт библиотек

In [ ]:
!pip install gensim -q
!pip install rank-bm25 -q

In [ ]:

import re
import random
import warnings
import os
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
import nltk
import numpy as np
import pandas as pd
import joblib

from tqdm.notebook import tqdm
from gensim.models import FastText
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import SnowballStemmer
from rank_bm25 import BM25Okapi

nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
# системные настройки
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")

### Объявление функций

In [ ]:
# функция для краткого обзора датасетов
def view_df(df):
    #display(df.head())
    df.info()
    display(df.columns)
    display(df.describe())

### Загрузка данных

In [ ]:
pth1 = '/content/Список_наших_товаров.csv'
pth2 = '/content/Сметченные_позиции_источник_1.csv'
pth3 = '/content/Сметченные_позиции_источник_2.csv'

if os.path.exists(pth1) & os.path.exists(pth2) & os.path.exists(pth3):
    goods = pd.read_csv(pth1, sep=',')
    match_1 = pd.read_csv(pth2, sep=',')
    match_2 = pd.read_csv(pth3, sep=',')
else:
    print('Something is wrong with loading data')

### Общая информация о датасете

In [ ]:
list_df = [goods, match_1, match_2]

for df in list_df:
    view_df(df)
    print('===================================================================================')
    print('===================================================================================')

**Вывод:** загружена и получена общая информация по датасетам:
- по датасету `goods`:
 - имеются 4 пропуска в наименовании товара
 - тип данных в столбцах соответствуют описанию
- по датасету `match_1`:
 - практически не заполнены столбцы *match_comment, validation_comment*
 - тип данных в столбцах соответствуют описанию          
- по датасету `match_2`:
 - практически не заполнен столбец *match_comment*
 - тип данных в столбцах соответствуют описанию  

Более подробный анализ, выявление дубликатов, ошибок заполнения и пр. будут произведены на следующих этапах обработки данных

##  Подготовка данных

### Обработка дубликатов

#### Датасет **goods**

In [ ]:
print('Исходный размер датасета goods:', goods.shape)
print(f'Количество дубликатов в наименовании или id товара: \
{(goods.duplicated(subset="sku_id") | goods.duplicated(subset="sku_name")).sum()}')

In [ ]:
# Исключим данные дубликаты
goods = goods[~(goods.duplicated(subset="sku_name"))]

In [ ]:
print('Размер датасета goods после удаления дубликатов:', goods.shape)

#### Датасет **match_1**

In [ ]:
print('Исходный размер датасета match_1:', match_1.shape)
print(f'Количество дубликатов с одинаковыми наименованиями товара-аналога и нашего товара: \
{match_1.duplicated(subset=["name", "item_name"]).sum()}')

In [ ]:
# Исключим данные дубликаты
match_1 = match_1.drop_duplicates(subset=["name", "item_name"], keep="first")

In [ ]:
print('Размер датасета match_1 после удаления дубликатов:', match_1.shape, '\n')
print('Количество записей, где одному нашему названию соответствуют несколько названий товаров-аналогов:'\
      , match_1.duplicated(subset=["item_name"]).sum())

#### Датасет **match_2**

In [ ]:
print('Исходный размер датасета match_2:', match_2.shape)
print('Количество дубликатов с одинаковыми наименованиями товара-аналога и нашего товара:'\
      , match_2.duplicated(subset=["product_name", "sku_id", "item_name"]).sum())

In [ ]:
# Исключим данные дубликаты
match_2 = match_2.drop_duplicates(subset=["product_name", "sku_id", "item_name"], keep="first")

In [ ]:
print('Размер датасета match_2 после удаления дубликатов:', match_2.shape, '\n')
print('Количество записей, где одному нашему названию соответствуют несколько названий товаров-аналогов:'\
      , match_2.duplicated(subset=["item_name"]).sum())

**Вывод:** Из 792 записей исходной тестовой выборки были удалены 39 дубликатов с одинаковыми наименованиями товара-аналога и нашего товара. Следует обратить внимание, что в оставшихся 753 записях содержаться 412 экземпляров, где одному нашему названию соответствуют несколько названий товаров-аналогов, т.е доля наших уникальных товаров составляет всего ~ 45% от общего количества.

### Исключение аналогов и объединение тестовой выборки

Для более точной оценки модели, оставим в тестовой выборке тип сопоставления товаров - `exact`

In [ ]:
match_1 = match_1.query('match_type == "exact"')
match_2 = match_2.query('match_type == "exact"')

Объединим датасеты `match_1`, `match_2` для получения общей тестовой выборки, оставив необходимые столбцы, которые приведем к одинаковым названиям

In [ ]:
match_1_ = match_1[['competitor_product_id', 'name', 'sku_id', 'item_name']]
match_1_.columns = ['competitor_id', 'competitor_name', 'vink_id', 'vink_name']

In [ ]:
match_2_ = match_2[['invoice_item_id', 'product_name', 'sku_id', 'item_name']]
match_2_.columns = ['competitor_id', 'competitor_name', 'vink_id', 'vink_name']

In [ ]:
test = pd.concat([match_1_, match_2_], ignore_index=True)
print('Размер тестовой выборки:', test.shape)

### Проверка содержания колонок датасетов

#### Датасет **train**

Переименуем датасет `goods` в `train`, оставив необходимые столбцы, которые приведем к одинаковым названиям

In [ ]:
train = goods[['sku_id', 'sku_name']]
train.columns = ['vink_id', 'vink_name']

In [ ]:
# очистим train от лишних и неинформативных данных
train = train[~train['vink_name'].isin(['ТЕСТ', 'v', 'test', 'тест', 'Наклейка',\
                                        'Образцы', 'Канцелярия', 'Этикетка'])]

# удалим записи с пропусками в столбце vink_name
train = train.dropna(subset=['vink_name']).reset_index(drop=True)

print('Размер тренировочной выборки:', train.shape)

#### Датасет **test**

In [ ]:
# выведем все значения в столбце vink_name, если они меньше 30 символов и не содержат цифр
filtered_values = test.loc[
    (test['vink_name'].str.len() < 30) &
    (~test['vink_name'].str.contains(r'\d', regex=True, na=False)),
    'vink_name'
]

print(filtered_values)

In [ ]:
# выведем все значения в столбце competitor_name, если они меньше 30 символов и не содержат цифр
filtered_values = test.loc[
    (test['competitor_name'].str.len() < 30) &
    (~test['competitor_name'].str.contains(r'\d', regex=True, na=False)),
    'competitor_name'
]

print(filtered_values)

In [ ]:
# очистим test от лишних и неинформативных данных
test = test[~test['competitor_name'].isin(['Создано пользователем', 'есрпви', "form.cleaned_data['name']"])]\
            .reset_index(drop=True)
test = test = test[test['vink_name'] != 'Уголок багетный с винтами']\
            .reset_index(drop=True)

In [ ]:
print('Размер тестовой выборки:', test.shape)

Проверим после очистки, содержатся ли все значения vink_name из датасета `test` в датасете `train`

In [ ]:
# Выведем названия из test, которых нет в train
missing_names = set(test['vink_name']) - set(train['vink_name'])
print(missing_names)

### Очистка и стемминг текста

На основе анализа состава названий товара, его описания, способов представления размеров и других параметров, морфологии наименований товаров и пр. предложены соответствующие подходящие задаче способы очистки текста, словарь, список стоп-слов и инструмент обработки - стемминг, который показал себя лучше лемматизатора, в связи с часто встречающимися сокращениями, вплоть до основы слова

In [ ]:
# Инициализация стеммера для русского языка
stemmer = SnowballStemmer(language='russian')

# Словарь для замены
replacement_dict = {
    "прозр": "прозрачный",
    "проз": "прозрачный",
    'vi': 'vilaseca'
}

# Список слов для удаления
stop_words = {'м', 'мм', 'мк', 'мкм', 'кг', 'г', 'для', 'на', 'с', 'и', 'плотностью'}

# Функция для обработки текста
def preprocess_text(text):
    text = str(text).lower()

    # Заменяем все нежелательные символы на пробелы
    text = re.sub(r'[^а-яёa-z0-9\s,\.]', ' ', text)

    # Заменяем точки и запятые на пробелы. Когда они находятся между цифрами без пробела - заменяем на точку
    text = re.sub(r'([^\d])[\.,]+([^\d])', r'\1 \2', text)
    text = re.sub(r'(\d)[\.,]+(\d)', r'\1.\2', text)

    # Отделяем буквы от цифр пробелом
    text = re.sub(r'(?<=\d)(?=[а-яa-z])|(?<=[а-яa-z])(?=\d)', ' ', text)

    # Заменяем 'x' или 'х' между цифрами на пробел
    text = re.sub(r'(\d)\s*[xх]\s*(\d)', r'\1 \2', text)

    # Удаляем числа, если после них идет "кг"
    text = re.sub(r'\b\d+(\.\d+)?\s*кг\b', ' ', text)

    # Удаление ведущих нулей у чисел
    text = re.sub(r'\b0+(\d+)\b', r'\1', text)

    # Убираем точки в конце строки
    text = re.sub(r'\.+$', '', text)

    # Удаляем стоп-слова
    text = " ".join(word for word in text.split() if word not in stop_words)

    # Замена слов по словарю
    text = " ".join([replacement_dict.get(word, word) for word in text.split()])

    # Применяем стемминг
    text = ' '.join([stemmer.stem(word) for word in text.split()])

    return text

**Вывод:** Проведена проверка содержания наименований в датасетах `train и test` на предмет ошибочных и неинформативных для задачи мэтчинга названий, сохранены необходимые столбцы, которые приведены к единым названиям, объединена тестовая выборка. Сформирована соответствующая задаче функция по очистке и итоговой подготовке датасетов `train и test` для дальнейшего обучения моделей и расчета метрик.

## Обучение моделей. 1 этап (Retriever)

Реализуем двухступенчатую архитектуру для решения задачи мэтчинга товаров. На 1-ом этапе создадим `retriever` -  для быстрого поиска топ-K кандидатов по запросу. На 2-ом этапе обучим модель `reranker` - для более точного переранжирования кандидатов из списка `retriever` с учётом контекста и морфологии наименований.

Реализуем BM25 для предварительного отбора кандидатов на 1 этапе

### Подготовка корпуса

In [ ]:
vink_names_all = train['vink_name'].tolist()
vink_ids_all = train['vink_id'].tolist()
corpus_bm25 = [word_tokenize(preprocess_text(name)) for name in vink_names_all]

In [ ]:
# Инициализация BM25
bm25 = BM25Okapi(corpus_bm25)

### Отбор кандидатов

In [ ]:
# Функция для отбора кандидатов из числа в списке k_top_list
def bm25_match_candidates(test_df, bm25, vink_names_all, vink_ids_all, k_top_list=[1, 5, 10]):

    max_k = max(k_top_list)
    matched_bm25 = test_df[['competitor_name', 'vink_name', 'vink_id']].copy()

    # Инициализация колонок для кандидатов
    for i in range(1, max_k + 1):
        matched_bm25[[f'vink_name_{i}', f'vink_id_{i}']] = None

    # Поиск кандидатов BM25
    for idx, row in matched_bm25.iterrows():
        query_tokens = word_tokenize(preprocess_text(row['competitor_name']))
        top_indices = np.argsort(bm25.get_scores(query_tokens))[::-1][:max_k]

        for i, top_idx in enumerate(top_indices, start=1):
            matched_bm25.at[idx, f'vink_name_{i}'] = vink_names_all[top_idx]
            matched_bm25.at[idx, f'vink_id_{i}'] = vink_ids_all[top_idx]

    # Подсчёт метрик acc@k
    for k in k_top_list:
        matched_bm25[f'is_matched@{k}'] = matched_bm25.apply(
            lambda row: int(row['vink_name'] in [row.get(f'vink_name_{i}') for i in range(1, k + 1)]),
            axis=1
        )
        print(f"Точность на модели BM25 (acc@{k}): {matched_bm25[f'is_matched@{k}'].mean():.4f}")

    return matched_bm25


Рассчитаем точность на 1 этапе для 1, 5 и 10 кандидатов из наших товаров соответственно

In [ ]:
matched_bm25 = bm25_match_candidates(
    test_df=test,
    bm25=bm25,
    vink_names_all=vink_names_all,
    vink_ids_all=vink_ids_all,
    k_top_list=[1, 5, 10]
)

Точность на 10 кандидатах уже достачно высокая, сохраним датафрейм с этими кандидатами для более точного ранжирования на следующем 2 этапе

In [ ]:
matched_bm25 = bm25_match_candidates(
    test_df=test,
    bm25=bm25,
    vink_names_all=vink_names_all,
    vink_ids_all=vink_ids_all,
    k_top_list=[10]
)

In [ ]:
#matched_bm25.head(3)

**Вывод:** реализована модель `BM25` для предварительного отбора кандидатов на 1 этапе. Точность подбора товаров заказчика к заданным товарам-аналогам на тестовом датасете на 1 этапе составили: **27.9%, 78.2% и 91.6%** соответственно для **1, 5 и 10** кандидатов из товаров Заказчика.   
В связи с достаточной точность на 10 кандидатах (свыше 90%) отберем этих кандидатов для более точного ранжирования на последующем 2 этапе

## Обучение модели. 2 этап (Reranker)

Реализуем модель `FastText` в качестве `reranker`. Модель показала себя наиболее подходящей для данного этапа в связи с возможностью работать с подсловами, обучением на основе n-грамм, учету морфологии и обработке редких слов

### Генерация синтетических наименований товаров для обучения модели

In [ ]:
# Функция для генерация синтетических наименований товаров
# Применяем функции для модификации наименований случайным образом и ровно 10 раз для каждого наименования
def remove_numbers(name):
    return re.sub(r'\d+', '', name).strip() or name

def keep_only_numbers(name):
    numbers = re.findall(r'\d+', name)
    return ' '.join(numbers) if numbers else name

def shuffle_words(name):
    words = name.split()
    random.shuffle(words)
    return ' '.join(words)

def remove_random_words(name):
    words = name.split()
    num_to_remove = random.randint(1, min(3, len(words)))
    for _ in range(num_to_remove):
        if words:
            words.pop(random.randint(0, len(words) - 1))
    return ' '.join(words)

def remove_english_words_and_shuffle(name):
    words = [word for word in name.split() if not re.match(r'[A-Za-z]+', word)]
    random.shuffle(words)
    return ' '.join(words)

def remove_random_numbers(name):
    words = name.split()
    words = [word for word in words if not word.isdigit() or random.random() > 0.5]
    return ' '.join(words)

def remove_russian_words_and_shuffle(name):
    words = [word for word in name.split() if not re.match(r'[А-Яа-я]+', word)]
    random.shuffle(words)
    return ' '.join(words)

def remove_numbers_and_shuffle(name):
    return shuffle_words(remove_numbers(name))

def keep_only_numbers_and_shuffle(name):
    return shuffle_words(keep_only_numbers(name))

def remove_random_numbers_and_shuffle(name):
    return shuffle_words(remove_random_numbers(name))

# Список всех функций
mod_functions = [
    remove_numbers,
    keep_only_numbers,
    shuffle_words,
    remove_random_words,
    remove_english_words_and_shuffle,
    remove_random_numbers,
    remove_russian_words_and_shuffle,
    remove_numbers_and_shuffle,
    keep_only_numbers_and_shuffle,
    remove_random_numbers_and_shuffle
]

# Генерация синтетических названий
def generate_synthetic_names(df, column):
    synthetic_data = []

    for name in tqdm(df[column].tolist(), desc="Генерация синтетических наименований"):
        sampled_funcs = random.choices(mod_functions, k=10)  # 10 случайных функций (с повторениями)
        for func in sampled_funcs:
            variant = func(name)
            if variant.strip():
                synthetic_data.append({'vink_name': name, 'vink_name_synt': variant})

    return pd.DataFrame(synthetic_data)

Генерация синтетического датафрейма

In [ ]:
train_synt = generate_synthetic_names(train, 'vink_name')

### Подготовка корпуса и обучение модели

In [ ]:
train_synt = pd.read_csv('synthetic_data.csv')

In [ ]:
# Функция для предобработки и токенизации корпуса на основе синтетического датасета
def create_corpus(df, col1, col2):
    corpus = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Создание корпуса"):
        combined_text = f"{row[col1]} {row[col2]}"
        preprocessed_text = preprocess_text(combined_text)
        tokenized_text = word_tokenize(preprocessed_text)
        corpus.append(tokenized_text)
    return corpus

In [ ]:
# Создание корпуса использования
corpus = create_corpus(train_synt, 'wink_name', 'wink_name_synt')

In [ ]:
# Фиксируем воспроизводимость результатов и обучаем модель
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Обучение FastText
class TqdmCorpus:
    def __init__(self, corpus):
        self.corpus = corpus

    def __iter__(self):
        for doc in tqdm(self.corpus, desc="Обучение FastText"):
            yield doc

embeddings_trained = FastText(
    sentences=TqdmCorpus(corpus),
    vector_size=200,
    window=5,
    epochs=5,
    seed=SEED,
    workers=1
).wv

In [ ]:
# Функция для подготовки эмбеддингов наименований для дальнейшего их семантического сравнения
def get_embedding(text, wv_embeddings, dim=None):
    """
    Преобразует текст в эмбеддинг, усредняя векторы слов.
    """
    if dim is None:
        dim = wv_embeddings.vector_size

    cleaned_text = preprocess_text(text)
    words = word_tokenize(cleaned_text)

    word_vectors = [wv_embeddings[word] for word in words if word in wv_embeddings]

    if not word_vectors:
        return np.zeros(dim)

    return np.mean(word_vectors, axis=0)

### Отбор финальных кандидатов мэтчинга и расчет метрики `accuracy`

In [ ]:
# Функция финального ранжирования кандидатов из 1 этапа
# в соответствии с заданным количеством кандидатов в списке n_top_list
def rerank_candidates_with_embeddings(
    matched_bm25, test, embeddings_model, dim, n_top_list=[1, 3, 5]
):

    max_k = max(n_top_list)

    # Создание итогового датафрейма
    matched_fin = test[['competitor_name', 'vink_id', 'vink_name']].copy()

    # Инициализация колонок под ранжированных кандидатов
    for i in range(1, max_k + 1):
        matched_fin[[f'vink_id_{i}', f'vink_name_{i}']] = None

    # Определение количества кандидатов из 1 этапа
    candidate_cols = sorted([
        int(col.split('_')[-1])
        for col in matched_bm25.columns
        if col.startswith('vink_name_')
    ])

    # Перебор всех строк и финальное ранжирование кандидатов
    for idx, row in matched_bm25.iterrows():
        comp_emb = get_embedding(row['competitor_name'], embeddings_model, dim).reshape(1, -1)

        candidates = []
        candidate_ids = []
        for i in candidate_cols:
            name_col = f'vink_name_{i}'
            id_col = f'vink_id_{i}'
            if pd.notna(row.get(name_col)):
                candidates.append(row[name_col])
                candidate_ids.append(row[id_col])

        if not candidates:
            continue

        # Сходства
        cand_embs = [get_embedding(name, embeddings_model, dim).reshape(1, -1) for name in candidates]
        sims = [cosine_similarity(comp_emb, emb)[0, 0] for emb in cand_embs]

        top_idxs = np.argsort(sims)[::-1][:max_k]
        for i, j in enumerate(top_idxs, 1):
            matched_fin.at[idx, f'vink_id_{i}'] = candidate_ids[j]
            matched_fin.at[idx, f'vink_name_{i}'] = candidates[j]

    # Подсчёт hits@k
    for k in n_top_list:
        matched_fin[f'is_matched@{k}'] = matched_fin.apply(
            lambda row: int(row['vink_name'] in [row.get(f'vink_name_{i}') for i in range(1, k + 1)]),
            axis=1
        )
        print(f"Итоговая точность (acc@{k}): {matched_fin[f'is_matched@{k}'].mean():.4f}")

    return matched_fin


In [ ]:
matched_fin = rerank_candidates_with_embeddings(
    matched_bm25=matched_bm25,
    test=test,
    embeddings_model=embeddings_trained,
    dim=embeddings_trained.vector_size,
    n_top_list=[1, 3, 5]
)

In [ ]:
#matched_fin.sample(3)

**Вывод:** реализована модель `FastText` для финального ранжирования кандидатов на 2 этапе. Точность подбора товаров заказчика к заданным товарам-аналогам на тестовом датасете на 2 этапе составили: **38.6%, 70.6% и 87%** соответственно для **1, 3 и 5** кандидатов из товаров Заказчика.   

<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Здорово, что ты изучаешь и применяешь технологии, наиболее подходящие под специфику задачи, у тебя получился отличный результат 🔥 </div>



## Реализация мэтчинга для отдельного запроса с товаром-аналогом

In [ ]:
def match_query(query_text, bm25_model, vink_names, embeddings_trained, k_top=10, n_top=5):
    # 1 Этап: отбор кандидатов моделью BM25
    query_text_clean = preprocess_text(query_text)
    query_tokens = word_tokenize(query_text_clean)

    scores = bm25_model.get_scores(query_tokens)
    top_indices = np.argsort(scores)[::-1][:k_top]

    bm25_matches = pd.DataFrame({
        'vink_name': [vink_names[i] for i in top_indices],
        'score': [round(scores[i], 4) for i in top_indices]
    })

    # 2 Этап: доранжирование кандидатов моделью FastText
    dim = embeddings_trained.vector_size
    query_embedding = get_embedding(query_text, embeddings_trained, dim).reshape(1, -1)

    candidate_embeddings = [
        get_embedding(name, embeddings_trained, dim).reshape(1, -1)
        for name in bm25_matches['vink_name']
    ]

    similarities = [
        cosine_similarity(query_embedding, emb)[0][0]
        for emb in candidate_embeddings
    ]

    bm25_matches['cosine_similarity'] = similarities

    top_candidates = bm25_matches.sort_values(
        by='cosine_similarity', ascending=False
    ).head(n_top)[['vink_name', 'cosine_similarity']]

    top_candidates.reset_index(drop=True, inplace=True)

    print('Запрос:', query_text)
    return top_candidates


In [ ]:
# Запрос для мэтчинга
query = "ПВХ Вспененный без маркировки, белый, 3 мм, стандартный размер"

# Отбор кандидатов
top_candidates = match_query(
    query_text=query,
    bm25_model=bm25,
    vink_names=vink_names_all,
    embeddings_trained=embeddings_trained,
    k_top=10,
    n_top=5
)

#top_candidates

## Выводы по проекту

**В рамках проекта был проведен:**     
- анализ и предобработка данных Заказчика, необходимая подготовка, очистка и стемминг, а также генерация синтетических данных для дальнейшего обучения.    
- обучение разных моделей и подбор оптимального пайплайна из моделей и их параметров для решения задачи мэтчинга.    
- проверка данных на тестовой выборке и расчет необходимых метрик.
_____________

**На этапе предобработки данных:**     
загружена и получена общая информация по датасетам Заказчика: имеются пропуски в данных, типы данных соответствует описанию.

**На этапе анализа и подготовки данных:**    
произведена обработка дубликатов в данных, объединена тестовая выборка. Проведена проверка содержания наименований в датасетах `train и test` на предмет некорректных названий.
На основе анализа состава наименований товара, его описания, способов представления размеров и других параметров, морфологии наименований товаров и пр. предложены соответствующие подходящие задаче способы очистки текста, словарь, список стоп-слов и инструмент обработки - стемминг, который показал себя лучше лемматизаторов (`Spacy, pymorphy2`), в связи с часто встречающимися сокращениями, вплоть до основы слова. Сформирована соответствующая задаче функция по очистке и итоговой подготовке датасетов `train и test` для дальнейшего обучения моделей и расчета метрик.

**На этапе обучения моделей:**    
для train датасета в целях обучения на парах "наименование" - "похожее наименование" сгенерирован синтетитический объем данных для товаров Заказчика, на основе их анализа и подходящих функций генерации.   
Нм этапе обучения было протестировано множество моделей и инструментов, которые применялись как для решения задачи в 1 этап, так и на разных этапах. Для первого этапа предварительного отбора рассматривались инструменты:
- `CountVectorizer`
- `TF-IDF Vectorizer`
- `Word2Vec`
- `BM25`    
и отдельно либо в комбинации с ними модели для второго этапа, более точного отбора и ранжирования:
- `NearestNeighbors` - c финальной метрикой acc@5 = 0.65
- `K-Means` с подбором количества кластеров, acc@5 = 0.71    
модели трасформеров   
- `CrossEncoder`, acc@5 = 0.72
- `Labse`, acc@5 = 0.74
- Бэггинг `FastText + LaBSE`,  acc@5 = 0,81  

В итоге лучшие результаты показала двухступенчатая архитектура для решения задачи мэтчинга товаров: на 1-ом этапе `BM25` -  для быстрого поиска топ-K кандидатов по запросу, на 2-ом этапе - модель `FastText` - для более точного переранжирования кандидатов из отобранного списка на 1 этапе.    
Преимущества комбинации `BM25 + FastText` в сочетании быстрого синтаксического фильтра и эффективного семантического ранжирования, где `BM25` классически работает на токенах и частоте слов, учитывая длину документа, покрывает редкие слова, числовые артефакты, аббревиатуры и опечатки, а `FastText` обучается на основе n-грамм, учитывая похожесть между словами, основанную на их морфологии.

**На этапе тестирования лучших моделей:**    
Точность подбора товаров заказчика к заданным товарам-аналогам на тестовом датасете     
на 1 этапе составили:
- **27.9%, 78.2% и 91.6%** соответственно для `1, 5 и 10` кандидатов из товаров Заказчика  

на 2 этапе после доранжирования из 10 кандидатов 1 этапа:
- **38.6%, 70.6% и 87%** соответственно для **1, 3 и 5** кандидатов из товаров Заказчика.

- прирост метрик составил **10,7% и 8,8%** соответственно для **1 и 5** кандидатов из товаров Заказчика

_________
*На основе проведенной работы по проекту можно сделать следующие рекомендации/предложения для решения Заказчиком задачи мэтчинга товаров:*
- продолжить накапливать базу смэтчинных товаров Заказчика с товарами-аналогами, для увеличения ее объема и разнообразия, так как ее объема крайне мало для полноценного обучения моделей и их проверки, с учетом соотношения в тестовой выборке уникальных наименований Заказчика к товарам-аналогам ~ 1:2
- с той же целью провести ревизию `аналогов`, товаров со статусами `matched`, `notified` для наполнения объема валидированных сметченных данных
- для упорядочивания в целом и повышения точности решения задачи мэтчинга предусмотреть при заведении в БД как собственных, так и товаров-аналогов, унифицированные блоки для однообразного внесения в БД ключевых характеристик товаров: `наименование, тип, размеры, производитель` и пр.
- предусмотреть невозможность внесения в БД явно ошибочных наименований, таких как: `'test', 'Создано пользователем', 'есрпви', 'form.cleaned_data['name']'` и пр.
- использовать для задачи мэтчинга поиск из 5 кандидатов, как дающий уверенный результат близкий к 90% совпадению, который также дает возможность анализировать более полный список предложенных товаров и выявлять причины, почему  истинный ответ на запрос не присутствует на 1-ом месте в списке.
- по мере наполнения базы, внесения необходимых изменений в наименования и подстройки модели, можно будет переходить к мэтчингу из меньшего списка кандидатов.
